In [2]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [9]:
#supress warnings
import warnings
warnings.filterwarnings("ignore")

Load the data:

In [3]:
df = pd.read_csv('Flickrs-Switzerland-1990-2017.csv')

Create a dataframe from locations:

In [6]:
df.head()

,userid,title,dateupload,datetaken,views,tags,lat,lon,place_id,woeid,location
0,134205948@N02,Il Tounot in veste autunnale visto da St Luc,1443166542,1990-01-02 00:00:00,485,winter january stluc valdanniviers fotoanalogi...,46.179830,7.604212,qgR3ESJWWrvegJA,781702.0,none
1,21053104@N03,SBB Re6/6 11633,1350361548,1990-01-01 15:26:36,1372,railroad switzerland ticino railway trains svi...,45.837485,9.020311,Ojl8iJtWWrhYxtw,782098.0,none
2,21053104@N03,SBB Ae6/6 11404,1247112224,1990-01-06 10:57:06,1501,railroad switzerland ticino railway trains svi...,46.194938,9.029774,eeFmDUFWWrttyWQ,781773.0,none
3,21053104@N03,SOB De4/4 21,1275845220,1990-01-21 11:44:57,1865,railroad switzerland railway trains zürich svi...,47.191068,8.686645,9kjHuh1WWr4tnqM,784014.0,none
4,71144572@N00,Plymouth Volare 1978 Taxi Zürich 007,1186944050,1990-01-23 00:46:32,1293,schweiz switzerland taxi zurich plymouth taxid...,47.472198,8.323516,y2rhZ5BTW7ltcr39fQ,29397634.0,none


In [10]:
df1 = df[['lat', 'lon']]
df1.lat = round(df1['lat'].astype(float),3)
df1.lon = round(df1['lon'].astype(float),3)
df1['location'] = 'none'
df1['location'][0:df1.shape[0]-1] = df1.iloc[0:df1.shape[0]-1, 0:2].apply(lambda x: str(x[0]) + ', ' + str(x[1]), axis=1)
len(df1.location.unique())

302784

As it can be seen, there are a lot of locations that are the same in the dataframe, so only unique locations are mapped to cantons and will be used later to create a dictionary (location to canton).

In [13]:
df1 = df1.drop_duplicates()

In [18]:
new = df1[['location']]
new['canton'] = 'none'
new.head()

,location,canton
0,"46.18, 7.604",none
1,"45.837, 9.02",none
2,"46.195, 9.03",none
3,"47.191, 8.687",none
4,"47.472, 8.324",none


The exc list is used to store the exceptions caused by timeout and l list - caused by 'too many requests' error.

In [20]:
exc = [] #to store timeout location
l = [] #to store exception batches

The function for mapping locations to cantons:

In [12]:
from time import sleep

def addCanton(row):
    loc = row
    try:
        sleep(0.01) # Time in seconds.
        geolocator = Nominatim()
        location = geolocator.reverse(loc)
        try:
            canton = location.raw["address"]["state"]
            return canton
        except KeyError:
            return None
    except GeocoderTimedOut:
        print("Exception: " + str(row))
        exc.append(loc)
        return 'none'

In [19]:
new.shape[0]

302784

In [ ]:
for i in range(0, 302500, 500):
    print("starting" + str(i))
    try:
        new['canton'][i:(i+500)] = new.iloc[i:(i+500), 0].apply(addCanton)
    except GeocoderServiceError:
        print("Exception with batch starting at "+str(i))
        l.append(i)
        sleep(600)
    sleep(300)

In [ ]:
#to be done when the previous cycle finished completely
new['canton'][302500:new.shape[0]] = new.iloc[302500:new.shape[0], 0].apply(addCanton)
#in case of GeocoderServiceError, which is usually caused by too many requests, wait some time and then try again

Now we need to deal with exceptions:

In [ ]:
l1 = []

In [ ]:
for i in l:
    print("starting" + str(i))
    try:
        new['canton'][i:(i+500)] = new.iloc[i:(i+500), 0].apply(addCanton)
    except GeocoderServiceError:
        print("Exception with batch starting at "+str(i))
        l1.append(i)
        sleep(600)
    sleep(300)

The other list l1 is used in case there are more exceptions, so they are stored separately. In case of new exceptions, the code above should be run again accordingly.

And now the remaining exceptions. The exc list can be used, but here we are just mapping the locations that are left (which should cover all the exc cases). 

In [ ]:
ind = df[df.canton == 'none'].index.values

In [ ]:
for i in ind:
    print("starting" + str(i))
    try:
        new['canton'][i:(i+1)] = new.iloc[i:(i+1), 0].apply(addCanton)
    except GeocoderServiceError:
        print("Exception at "+str(i))
        sleep(600)

In case of any exceptions the code above (two cells) should be run again. At the end there should be no elements in the dataframe where canton is 'none'. Here's the check:

In [ ]:
#The result should be empty
new[new.canton == 'none']

Now it's important to save the dataframe, so it can be used later without redoing the whole process again.

In [ ]:
with open('df3.pickle', 'wb') as handle:
    pickle.dump(new, handle, protocol=pickle.HIGHEST_PROTOCOL)

The mapped dataframe is stored in df3.pickle now.